In [1]:

# ── 1. Imports ────────────────────────────────────────────────────────────────
import os, sys, math, json
from pathlib import Path
from datetime import datetime, timedelta

import cdsapi                 # Climate Data Store client
import xarray as xr           # convenient NetCDF handling
import numpy as np
import pandas as pd

# Optional but handy for solar elevation:
try:
    from astral.sun import elevation as solar_elevation
except ImportError:
    !pip -q install astral
    from astral.sun import elevation as solar_elevation

# ── 2. Constants ──────────────────────────────────────────────────────────────
LAT, LON = 55.587444, 13.019811 #59.329, 18.068          # Stockholm Central Station  (decimal °)  :contentReference[oaicite:6]{index=6}
# 0.125-degree buffer around Stockholm Central
BUFFER = 0.01
  
north = round(LAT + BUFFER, 2)   # → 59.45
west  = round(LON - BUFFER, 2)   # → 17.94
south = round(LAT - BUFFER, 2)   # → 59.20
east  = round(LON + BUFFER, 2)   # → 18.19

BOX = [north, west, south, east]

# ── Helper to build a dated filename ──────────────────────────────────────────
def _make_target(base: str, start, end, ext="nc") -> str:
    """
    Return e.g.  base_20240101_20250101.nc
    """
    start_str = pd.to_datetime(start).strftime("%Y%m%d")
    end_str   = pd.to_datetime(end).strftime("%Y%m%d")
    return f"{base}_{start_str}_{end_str}.{ext}"
# ── 3. ERA5 downloader ────────────────────────────────────────────────────────
def download_era5_point(start, end, base="ERA5_malmö/era5_malmö"):
    target = _make_target(base, start, end)          # <── new line

    """
    Download ERA5 single-level fields for a point (small box) and time span.
    start, end : ISO strings 'YYYY-MM-DD' or datetime objects (inclusive)
    """
    start = pd.to_datetime(start)
    end   = pd.to_datetime(end)

    years  = sorted({d.year  for d in pd.date_range(start, end, freq='D')})
    months = sorted({d.month for d in pd.date_range(start, end, freq='D')})
    days   = sorted({d.day   for d in pd.date_range(start, end, freq='D')})

    hours  = [f"{h:02d}:00" for h in range(24)]          # full diurnal cycle

    c = cdsapi.Client()                                  # requires ~/.cdsapirc

    c.retrieve(
        "reanalysis-era5-single-levels",                 # dataset id:contentReference[oaicite:7]{index=7}
        {
          "product_type" : "reanalysis",
          "variable"     : [
              "10m_u_component_of_wind",
              "10m_v_component_of_wind",
              "total_cloud_cover"#,
              #"surface_net_solar_radiation"
          ],
          "year"  : [f"{y}"        for y in years ],
          "month" : [f"{m:02d}"    for m in months],
          "day"   : [f"{d:02d}"    for d in days  ],
          "time"  : hours,
          "area"  : BOX,           # N W S E in degrees
          "format": "netcdf"
        },
        target
    )
    return Path(target)


In [9]:
# ── Prerequisite imports ────────────────────────────────────────────────
import os, zipfile, tempfile
import numpy as np
import pandas as pd
import xarray as xr
from astral import Observer
from astral.sun import elevation as solar_elevation

# Stockholm Central coords
LAT, LON = 59.329, 18.068

def open_era5_with_stability(path):
    """
    Reads an ERA5 download (zip/netcdf/grib), selects the
    nearest point to Stockholm Central, computes wind speed,
    wind direction, and Pasquill–Gifford stability (A–F), and
    returns an xarray.Dataset.
    """
    # 1. Detect file type (magic number)
    with open(path, "rb") as f:
        sig = f.read(4)
    # 2. If ZIP, extract the first .nc inside
    if sig.startswith(b"PK\x03\x04"):
        with zipfile.ZipFile(path, "r") as z:
            nc_files = [fn for fn in z.namelist() if fn.lower().endswith(".nc")]
            if not nc_files:
                raise FileNotFoundError("No .nc file inside archive.")
            tmpdir    = tempfile.mkdtemp()
            extracted = z.extract(nc_files[0], path=tmpdir)
            nc_path   = extracted
    else:
        nc_path = path

    # 3. Choose engine
    engine = "cfgrib" if sig.startswith(b"GRIB") else "netcdf4"

    # 4. Open dataset
    ds = xr.open_dataset(nc_path, engine=engine)

    # 5. Rename valid_time→time if present
    if "valid_time" in ds.coords:
        ds = ds.rename({"valid_time": "time"})

    # 6. Subset to nearest Stockholm Central
    ds = ds.sel(latitude=LAT, longitude=LON, method="nearest")

    # 7. Wind speed & direction
    ds["wind_speed"]     = np.hypot(ds["u10"], ds["v10"])
    ds["wind_direction"] = (270 - np.degrees(np.arctan2(ds["v10"], ds["u10"]))) % 360

    # 8. Pasquill–Gifford (A–F) via solar elevation + cloud cover
    # 8.1 Prepare inputs
    times = pd.to_datetime(ds["time"].values)
    observer = Observer(latitude=LAT, longitude=LON, elevation=0)
    elev      = np.array([solar_elevation(observer, t) for t in times])
    ws        = ds["wind_speed"].values
    tcc       = ds["tcc"].values.squeeze()  # cloud cover 0–1


    # 8.3 Full Pasquill–Gifford lookup driven by the table in the screenshot
    def pg(w, elev_deg, cloud):
        """
        Parameters
        ----------
        w         : wind speed at 10-m height (m s-1)
        elev_deg  : solar elevation angle (°) – positive by day, negative at night
        cloud     : total cloud cover (fraction 0–1)

        Returns
        -------
        str  • stability class A-F
        """

        # --- Helper: bucket insolation by solar elevation ---------------
        # Strong  >60°, Moderate 35–60°, Weak <35°
        if elev_deg > 60:
            ins_cat = "strong"
        elif elev_deg > 35:
            ins_cat = "moderate"
        elif elev_deg > 0:
            ins_cat = "weak"
        else:
            ins_cat = None            # night

        # --- DAYTIME branch --------------------------------------------
        if ins_cat is not None:
            if   w < 2:                       # <2 m/s
                return "A" if ins_cat in ("strong", "moderate") else "B"
            elif w < 3:                       # 2–3
                if   ins_cat == "strong":     return "A"
                elif ins_cat == "moderate":   return "B"
                else:                         return "C"
            elif w < 4:                       # 3–4
                if   ins_cat == "strong":     return "B"
                elif ins_cat == "moderate":   return "B"  # choose the lower value of B-C
                else:                         return "C"
            elif w < 6:                       # 4–6
                if   ins_cat == "strong":     return "C"
                elif ins_cat == "moderate":   return "C"  # lower value of C-D
                else:                         return "D"
            else:                             # >6
                if   ins_cat == "strong":     return "C"
                elif ins_cat == "moderate":   return "D"
                else:                         return "D"

        # --- NIGHT-TIME branch -----------------------------------------
        # Cloud categories from table:
        #   • 'cloudy'  : thin clouds OR >4/8 low cloud  → cloud ≥ 0.5
        #   • 'clear'   : ≤3/8 low cloud                → cloud < 0.5
        cloudy = cloud >= 0.5

        if cloudy:                    # “Tunna moln eller >4/8 låga”
            if   w < 2:  return "F"   # table shows F* – treat as F
            elif w < 3:  return "E"
            else:       return "D"    # ≥3 m/s all map to D
        else:                         # “≤3/8 låga”
            if   w < 2:  return "F"   # F* in table
            elif w < 3:  return "F"
            elif w < 4:  return "E"
            else:       return "D"


    pg_vec = np.vectorize(pg)
    classes = pg_vec(ws, elev, tcc)

    ds["stability_class"] = xr.DataArray(
        classes,
        coords={"time": ds["time"].values},
        dims=["time"]
    )

    # 9. Cleanup if we unzipped
    if sig.startswith(b"PK\x03\x04"):
        try:
            os.remove(nc_path)
            os.rmdir(tmpdir)
        except OSError:
            pass

    return ds



In [22]:
pd.to_datetime("2024-01-01")

Timestamp('2024-01-01 00:00:00')

In [32]:
# Retrieve one year and let the helper name the file
nc_file = download_era5_point("2024-06-01", "2024-12-31")
print("Saved →", nc_file)          # ➞ era5_stockholm_20240101_20250101.nc



2025-07-14 14:55:30,185 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-14 14:55:30,900 INFO Request ID is bc85c745-b591-4ead-b297-b5d85301e086
2025-07-14 14:55:31,005 INFO status has been updated to accepted
2025-07-14 14:55:45,337 INFO status has been updated to running
2025-07-14 15:07:52,709 INFO status has been updated to successful
                                                                                      

Saved → era5_stockholm_20240601_20241231.nc


In [31]:
ds = open_era5_with_stability("era5_stockholm_20240801_20241231.nc")
print(ds[['wind_speed','wind_direction','stability_class']].to_dataframe().head())

                     wind_speed  wind_direction stability_class  number  \
time                                                                      
2024-08-01 00:00:00    3.480160      322.183472               E       0   
2024-08-01 01:00:00    3.476162      325.791351               E       0   
2024-08-01 02:00:00    3.510871      324.129303               E       0   
2024-08-01 03:00:00    3.693168      321.799805               C       0   
2024-08-01 04:00:00    3.857137      322.685120               C       0   

                     latitude  longitude expver  
time                                             
2024-08-01 00:00:00     59.32      18.06   0001  
2024-08-01 01:00:00     59.32      18.06   0001  
2024-08-01 02:00:00     59.32      18.06   0001  
2024-08-01 03:00:00     59.32      18.06   0001  
2024-08-01 04:00:00     59.32      18.06   0001  


In [ ]:
# ── Monthly downloader ──────────────────────────────────────────────────────
from pathlib import Path
import pandas as pd

def download_monthly_series(start_year: int, end_year: int,
                            base_dir: str = "ERA5_malmo",
                            base_name: str = "era5_malmo"):
    """
    Loop over every month between start_year and end_year (inclusive),
    download that month, and save to <base_dir>/<base_name>_YYYYMMDD_YYYYMMDD.nc
    """
    out_dir = Path(base_dir)
    out_dir.mkdir(exist_ok=True)

    # Build a monthly date index (1st of each month)
    dates = pd.date_range(f"{start_year}-01-01", f"{end_year}-12-31", freq="6MS")

    for start in dates:
        # last day of the same month
        end = (start + pd.offsets.MonthEnd(6)).date()
        start_str = start.strftime("%Y-%m-%d")
        end_str   = end.strftime("%Y-%m-%d")

        target = out_dir / _make_target(base_name, start_str, end_str)

        if target.exists():
            print(f"✔ {target.name} already exists – skipping")
            continue

        print(f"⇓ Downloading {start_str} → {end_str} …")
        download_era5_point(start_str, end_str)


In [4]:

# Example: grab Jan 2023 → Dec 2024
download_monthly_series(2024, 2024)


⇓ Downloading 2024-01-01 → 2024-06-30 …


2025-07-18 09:57:16,043 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-18 09:57:16,673 INFO Request ID is f79c591d-edf3-4aa7-a791-24ae23080346
2025-07-18 09:57:16,767 INFO status has been updated to accepted
2025-07-18 09:57:30,894 INFO status has been updated to running
2025-07-18 10:07:37,470 INFO status has been updated to successful


⇓ Downloading 2024-07-01 → 2024-12-31 …


2025-07-18 10:07:39,304 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-18 10:07:39,917 INFO Request ID is 3196082f-2e1b-4fa6-b02c-d5008642be79
2025-07-18 10:07:39,992 INFO status has been updated to accepted
2025-07-18 10:07:48,633 INFO status has been updated to running
2025-07-18 10:20:02,081 INFO status has been updated to successful


In [11]:
from pathlib import Path
import pandas as pd

def combine_monthly_to_csv(
    base_dir: str = "ERA5_files",
    base_name: str = "era5_stockholm",
    csv_name:  str = "era5_stockholm_combined.csv",
    hourly_qc: bool = True,          # turn on financial-style gap check
    dup_policy: str = "last"         # or "first" or "mean"
):
    """
    Load all <base_name>_*.nc files in <base_dir> (monthly ERA5 downloads),
    derive wind_speed / wind_direction / stability_class via open_era5_with_stability,
    concatenate into one time-indexed DataFrame, optionally enforce a continuous
    hourly index (QA), and write to CSV.
    """
    data_dir = Path(base_dir)
    files = sorted(data_dir.glob(f"{base_name}_*.nc"))
    if not files:
        raise FileNotFoundError(f"No files matching {base_name}_*.nc in {base_dir}")

    frames = []
    for f in files:
        print(f"⇢ Loading {f.name}")
        ds = open_era5_with_stability(str(f))  # use your existing function
        df = ds[["wind_speed", "wind_direction", "stability_class"]].to_dataframe()
        frames.append(df)

    # Concatenate
    big_df = pd.concat(frames)

    # Ensure sorted by time
    big_df = big_df.sort_index()

    # Handle duplicate timestamps (can happen if date ranges overlap)
    if dup_policy == "last":
        big_df = big_df[~big_df.index.duplicated(keep="last")]
    elif dup_policy == "first":
        big_df = big_df[~big_df.index.duplicated(keep="first")]
    elif dup_policy == "mean":
        big_df = big_df.groupby(level=0).mean(numeric_only=True)
    else:
        raise ValueError("dup_policy must be 'last', 'first', or 'mean'.")

    # Optional QA step (financial risk–style: enforce complete grid)
    if hourly_qc:
        full_idx = pd.date_range(big_df.index.min(), big_df.index.max(), freq="H")
        big_df = big_df.reindex(full_idx)  # gaps appear as NaN; no forward fill by default

    # Save
    out_csv = data_dir / csv_name
    big_df.to_csv(out_csv, index_label="time")
    print(f"✓ Saved combined CSV → {out_csv}")

    return big_df

# EXAMPLE: combine everything already in ERA5_files/
combined_df = combine_monthly_to_csv(
    base_dir="ERA5_malmo",
    base_name="era5_malmo",
    csv_name="era5_malmo_2024_combined.csv",
    hourly_qc=True,
    dup_policy="last"
)

combined_df.head()


⇢ Loading era5_malmo_20240101_20240630.nc


c:\Users\johan\miniconda3\envs\QRA_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⇢ Loading era5_malmo_20240701_20241231.nc
✓ Saved combined CSV → ERA5_malmo\era5_malmo_2024_combined.csv


C:\Users\johan\AppData\Local\Temp\ipykernel_16680\3537774544.py:47: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_idx = pd.date_range(big_df.index.min(), big_df.index.max(), freq="H")


,wind_speed,wind_direction,stability_class,number,latitude,longitude,expver
2024-01-01 00:00:00,4.800020,145.529572,D,0,55.58,13.01,0001
2024-01-01 01:00:00,4.163002,143.631607,D,0,55.58,13.01,0001
2024-01-01 02:00:00,4.032418,147.548813,D,0,55.58,13.01,0001
2024-01-01 03:00:00,3.860020,149.572540,D,0,55.58,13.01,0001
2024-01-01 04:00:00,3.624707,149.601837,D,0,55.58,13.01,0001
